## Contributors
### Jay Upadhyay
### Haruki Miyazaki
###
###
###

# Libraries

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
import matplotlib.pyplot as plt
import random
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
df_orig=pd.read_csv("adult.csv")
df_orig.dropna(inplace=True)
df_orig

# 1. Project Proposal


We pulled the data from https://archive-beta.ics.uci.edu/dataset/2/adult .

# 2. Data Collection and data cleaning


## Changing workclass to numeric

In [ ]:
workclass_list = list(sorted(set(df_orig[" workclass"])))

for i,x in enumerate(workclass_list):
    print(i,":", x)
    
df_orig[' workclass'].replace(workclass_list,
                        range(len(workclass_list)), inplace=True)

## Changing Education to Numeric

In [ ]:
df_orig['education'].unique()

In [ ]:
education_list=[ ' Preschool',' 1st-4th',' 5th-6th',' 7th-8th',' 9th',' 10th',' 11th',' 12th',' HS-grad',
                              ' Assoc-voc',' Assoc-acdm',' Some-college',' Bachelors', ' Masters',
                              ' Prof-school' ,' Doctorate']

df_orig['education'].replace(education_list,
                        range(16), inplace=True)

df_orig
for x in range(16):
    print(x,":", education_list[x])

#### Note. I want to come back to this and bin it into appropriate 
I also chose to exclude this for now because I believe education-num already represents it numerically.

## Changing marital status to numeric

In [ ]:
marital_list = list(sorted(set(df_orig["marital-status"])))

for i,x in enumerate(marital_list):
    print(i,":", x)

df_orig['marital-status'].replace(marital_list,
                        range(len(marital_list)), inplace=True)

## Occupation

I don't like the ? mark section.

In [ ]:
occupation_list = list(sorted(set(df_orig["occupation"])))

for i,x in enumerate(occupation_list):
    print(i,":", x)

df_orig['occupation'].replace(occupation_list,
                        range(len(occupation_list)), inplace=True)

## Relationship conversion to numeric

I don't like other relative section here.


In [ ]:
relationship_list = list(sorted(set(df_orig["relationship"])))

for i,x in enumerate(relationship_list):
    print(i,":", x)

df_orig['relationship'].replace(relationship_list,
                        range(len(relationship_list)), inplace=True)

## Race conversion to numeric


In [ ]:
race_list = list(sorted(set(df_orig["race"])))

for i,x in enumerate(race_list):
    print(i,":", x)

df_orig['race'].replace(race_list,
                        range(len(race_list)), inplace=True)

## Sex conversion to numeric

In [ ]:
sex_list = list(sorted(set(df_orig["sex"])))

for i,x in enumerate(sex_list):
    print(i,":", x)

df_orig['sex'].replace(sex_list,
                        range(len(sex_list)), inplace=True)

## Country to Numeric

In [ ]:
# Changing a column "class" to be 0:<=50K and 1:>50K
country_list = list(sorted(set(df_orig["native-country"])))

for i,x in enumerate(country_list):
    print(i,":", x)

df_orig['native-country'].replace(country_list,
                        range(len(country_list)), inplace=True)

## Income to numeric

In [ ]:
# Changing a column "class" to be 0:<=50K and 1:>50K
class_list = list(sorted(set(df_orig["class"])))

for i,x in enumerate(class_list):
    print(i,":", x)
    
df_orig['class'].replace(class_list,
                        range(len(class_list)), inplace=True)

In [ ]:
df_orig

## Correlation Matrix

In [ ]:

df_small = df_orig.iloc[:,[0,1,2,3,4,5,6,14]]
correlation_mat = df_small.corr()
sns.heatmap(correlation_mat, annot = True)
plt.show()




In [ ]:
df_small = df_orig.iloc[:,[7,8,9,10,11,12,13,14]]
correlation_mat = df_small.corr()
sns.heatmap(correlation_mat, annot = True)
plt.show()


We see that age, education, marital status, relationship, sex, capital gain, capital loss, hours per week are all correlated to class(our income variable). The rest are relatively low so ignored for now.


# 3. EDA

# 4. Techniques Used

## KNN First Attempt

In [ ]:
# Splits rows to create a train and test
train,test=train_test_split(df_orig.iloc[:,[4,8,9,14]],test_size=0.33)
#train,test=train_test_split(df_orig.iloc[:,[0,1,4,5,6,7,8,9,12,13,14]],test_size=0.33)
X_train=train.iloc[:,:3] #Take train portion and first columns
Y_train=train.iloc[:,3:].values.ravel() #target we are trying to predict for
X_test=test.iloc[:,:3] #take test portion columns 
Y_test=test.iloc[:,3:].values.ravel()


In [ ]:
knn=KNeighborsClassifier(
    n_neighbors=80, 
    weights='uniform', 
    algorithm='auto', 
    leaf_size=30, 
    p=2, 
    #metric='matching', 
    metric='minkowski', 
    metric_params=None, 
    n_jobs=None)

In [ ]:
knn.fit(X_train,Y_train)

In [ ]:
pred_test=knn.predict(X_test)
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(Y_test, pred_test))
print("Precision:",metrics.precision_score(Y_test, pred_test))

In [ ]:
display = metrics.PrecisionRecallDisplay.from_predictions(Y_test, pred_test, name="kNN")
display.ax_.set_title("2-class Precision-Recall curve")

In [ ]:
precision, recall, thresholds = metrics.precision_recall_curve(Y_test,pred_test)
precision

In [ ]:
metrics.roc_curve(Y_test,pred_test)

## Second Attempt Random Forest 



In [ ]:
# Splits rows to create a train and test
#0,4,7,9,10,11,12
#0,1,3,5,6,7,8,9,12,13,14 DO NOT DELETE, CAN USE THIS LATER FOR IMAGE/SLIDES
#[0,1,4,6,7,8,9,10,11,12,14]] 85%
#168
train,test=train_test_split(df_orig.iloc[:,[0,1,4,6,7,8,9,10,11,12,14]],test_size=0.2)
X_train=train.iloc[:,:10].to_numpy() #Take train portion and first columns
Y_train=train.iloc[:,10:].values.ravel() #target we are trying to predict for
X_test=test.iloc[:,0:10].to_numpy()#take test portion columns 
Y_test=test.iloc[:,10:].values.ravel()

In [ ]:
rf = RandomForestClassifier(n_estimators=100,
                            # criterion='gini', 
                            max_depth=None, 
                            # min_samples_split=2, 
                            # min_samples_leaf=1, 
                            # min_weight_fraction_leaf=0.0,
                            # max_features='sqrt',
                            # max_leaf_nodes=None, 
                            # min_impurity_decrease=0.0, 
                            # bootstrap=True, 
                            # oob_score=False, 
                            # n_jobs=None, 
                            # random_state=None, 
                            # verbose=0,
                            # warm_start=False, 
                            # class_weight=None,
                            # ccp_alpha=0.0, 
                            # max_samples=None
                            )
rf.fit(X_train,Y_train)


In [ ]:
pred_test=rf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(Y_test, pred_test))
print("Precision:",metrics.precision_score(Y_test, pred_test))

# 5. Data Processing Part 2

In [ ]:
df_features=df_orig.iloc[:,[0,1,4,6,7,8,9,10,11,12,14]].copy()
df_features

In [ ]:
#max(normalize( df_features["capital-gain"].to_numpy().reshape(1,-1)))
#scaler=StandardScaler()
##scaler.fit()
#scaler.fit_transform(df_features["capital-gain"].to_numpy().reshape(1,-1)).reshape(df_features["capital-gain"].shape)
#scaler.fit_transform(X_train.reshape(-1,X_train.shape[-1])).reshape(X_train.shape)
#df_features

#df_features[["capital-gain", "capital-loss"]] = StandardScaler.fit_transform(X=df_features[["capital-gain", "capital-loss"]],y=df_features[["capital-gain", "capital-loss"]])
df_features[["capital-gain"]]=df_features[["capital-gain"]].astype(float)
df_features[["capital-loss"]]=df_features[["capital-loss"]].astype(float)
# Min-Max Normalization

cap_gain= df_features[["capital-gain", "capital-loss"]] 

df_features=df_features.drop("capital-gain", axis=1)
df_features=df_features.drop("capital-loss", axis=1)
#df_features.drop("capital-loss", axis=1)
df_norm = (cap_gain-cap_gain.min())/(cap_gain.max()-cap_gain.min())
df_features = pd.concat((df_norm, df_features), 1)
 
# print("Scaled Dataset Using Pandas")
# df_norm.head()

df_features 

In [ ]:
df_features.insert(loc=9, column='loss/gain', value=0)
df_features['loss/gain'] = np.where(((df_features['capital-loss']==0) & (df_features['capital-gain']==0)), 0, 1)
df_features

In [ ]:
df_features.insert(loc=10, column='Age Group', value=0)
df_features['Age Group'] = pd.cut(
    df_features['Age'], 
    [0,29, 44, 60,100], 
    labels=[0, 1,2,3]
)
df_features["Age Group"]=pd.to_numeric(df_features["Age Group"])


We want to modify these values and see if we can increase correlation. 
We employ methods such as normalization and binning.

In [ ]:
df_small = df_features.iloc[:,:]

correlation_mat = df_small.corr()
sns.heatmap(correlation_mat, annot = True)
plt.show()


## Models Round 2

In [ ]:
# df_features=df_features.drop("capital-gain", axis=1)
# df_features=df_features.drop("capital-loss", axis=1)
df_features=df_features.drop("Age", axis=1)
train,test=train_test_split(df_features,test_size=0.2)
X_train=train.iloc[:,:11].to_numpy() #Take train portion and first columns
Y_train=train.iloc[:,11:].values.ravel() #target we are trying to predict for
X_test=test.iloc[:,0:11].to_numpy()#take test portion columns 
Y_test=test.iloc[:,11:].values.ravel()
rf = RandomForestClassifier(n_estimators=100,
                            # criterion='gini', 
                            max_depth=None, 
                            # min_samples_split=2, 
                            # min_samples_leaf=1, 
                            # min_weight_fraction_leaf=0.0,
                            # max_features='sqrt',
                            # max_leaf_nodes=None, 
                            # min_impurity_decrease=0.0, 
                            # bootstrap=True, 
                            # oob_score=False, 
                            # n_jobs=None, 
                            # random_state=None, 
                            # verbose=0,
                            # warm_start=False, 
                            # class_weight=None,
                            # ccp_alpha=0.0, 
                            # max_samples=None
                            )
rf.fit(X_train,Y_train)
pred_test=rf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(Y_test, pred_test))
print("Precision:",metrics.precision_score(Y_test, pred_test))
#.85659450
#0.8588975894365116 after removing age

In [ ]:
indexes = rf.feature_importances_.argsort()
plt.barh(df_features.columns[indexes], rf.feature_importances_[indexes])
plt.xlabel("Random Forest Feature Importance")

# 6. Project Proposal